In [ ]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    env is more like a env_model (not env) to return you probs 
    on different states and rewards. 
    """
    
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    while True:
        
        # delta to record Value:V(s) diff, determines when to stop
        delta = 0
        
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = []
            
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    
                    # Calculate the expected value. 
                    v.append(action_prob * prob * 
                             (reward + discount_factor * V[next_state]))
                    
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            
            V[s] = sum(v)
        # Stop evaluating once our value function change
        # is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [ ]:
# helper function
def one_step_lookahead(env, state, V):
    """
    Helper function to calculate the value 
    for all action in a given state.

    Args:
        state: state id
        V: The value to use as an estimator, Vector of length env.nS

    Returns:
        A vector of length env.nA containing 
        the expected value of each action: q(s, a)
    """
    A = np.zeros(env.nA)
    for a in range(env.nA):
        for prob, next_state, reward, done in env.P[state][a]:
            A[a] += prob * (reward + discount_factor * V[next_state])
    return A

In [ ]:
def policy_improvement(env, 
                       policy_eval_fn=policy_eval, 
                       discount_factor=1.0):
    
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        # Evaluate the current policy
        V = policy_eval_fn(policy, env, discount_factor)
        
        # Helper variable
        # Will be set to false if we make any changes to the policy
        policy_stable = True
        
        for s in range(env.nS):
            # The most possible action 
            # would take under the currect policy
            # which is also the best action we believe last iteration
            chosen_a = np.argmax(policy[s])
            
            # Find the best action in this iteration
            # by one-step lookahead
            # Ties are resolved arbitarily
            action_values = one_step_lookahead(env, s, V)
            best_a = np.argmax(action_values)
            
            # Greedily update the policy
            if chosen_a != best_a:
                policy_stable = False
            
            # to choose best_a as next policy for s
            # ! mark the implementation here
            policy[s] = np.eye(env.nA)[best_a]
        
        # If the policy is stable we've found an optimal policy. 
        # Return it
        if policy_stable:
            return policy, V

In [ ]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):

    V = np.zeros(env.nS)
    while True:
        # helper variable: Stopping condition
        delta = 0
        
        # Update each state...
        for s in range(env.nS):
            # Do a one-step lookahead to find the best action
            A = one_step_lookahead(env, s, V)
            best_action_value = np.max(A)
            
            # Calculate delta across all states seen so far
            delta = max(delta, np.abs(best_action_value - V[s]))
            
            # Update the value function. Ref: Sutton book eq. 4.10. 
            V[s] = best_action_value
            
        # Check if we can stop 
        if delta < theta:
            break
    
    # Create a deterministic policy using V
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        # One step lookahead to find the best action for this state
        A = one_step_lookahead(s, V)
        best_action = np.argmax(A)
        # Always take the best action
        policy[s, best_action] = 1.0
        # or policy[s] = np.eye(env.nA)[best_a]
    
    return policy, V

In [1]:
# excercise

In [ ]:
def gamblers_one_step_lookahead(rewards, s, V):
    """
    Helper function to calculate the value for all action in a given state.

    Args:
        s: The gambler’s capital. Integer.
        V: The vector that contains values at each state. 
        rewards: The reward vector.

    Returns:
        A vector containing the expected value of each action. 
        Its length equals to the number of actions.
    """
    A = np.zeros(101)
    stakes = range(1, min(s, 100-s)+1) # Your minimum bet is 1, maximum bet is min(s, 100-s).
    for a in stakes:
        # rewards[s+a], rewards[s-a] are immediate rewards.
        # V[s+a], V[s-a] are values of the next states.
        # This is the core of the Bellman equation: The expected value of your action is 
        # the sum of immediate rewards and the value of the next state.
        A[a] = p_h * (rewards[s+a] + V[s+a]*discount_factor) + (1-p_h) * (rewards[s-a] + V[s-a]*discount_factor)
    return A